In [25]:
""" 
For scraping review links from a general company search on Glassdoor 
Use other tools for scraping review content 
"""

' \nFor scraping review links from a general company search on Glassdoor \nUse other tools for scraping review content \n'

In [3]:
import pandas as pd
import requests
import time
import math
import re
import pickle
import itertools
import glob

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [32]:
# add chrome options
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--headless")

# initiate driver 
driver =  webdriver.Chrome(options=chrome_options) 
page_num=1
url = f'https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=1&page={page_num}&locId=1132348&locType=C&locName=New%20York,%20NY%20(US)&sector=10010'
driver.get(url) 

htmlSource = driver.page_source
soup = BeautifulSoup(htmlSource,"html.parser")

num_pages = get_num_pages(soup)
print(f'There are {num_pages} total pages.')

all_links = []
max_rating = 3

# get links from first page if any companies lower than max rating 
get_links_page(soup, all_links, max_rating)

if num_pages > 1:
    for i in range(2,num_pages+1,1):
        print(f'Getting results for page {i}')
        url = f'https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=1&page={i}&locId=1132348&locType=C&locName=New%20York,%20NY%20(US)&sector=10010'
        driver.get(url)
        wait_for_page(driver)
        htmlSource = driver.page_source
        soup = BeautifulSoup(htmlSource,"html.parser")
        get_links_page(soup, all_links, max_rating)
all_links

There are 384 total pages.
Getting results for page 2
Getting results for page 3
Getting results for page 4
Getting results for page 5
Getting results for page 6
Getting results for page 7
Getting results for page 8
Getting results for page 9
Getting results for page 10
Getting results for page 11
Getting results for page 12
Getting results for page 13
Getting results for page 14
Getting results for page 15
Getting results for page 16
Getting results for page 17
Getting results for page 18
Getting results for page 19
Getting results for page 20
Getting results for page 21
Getting results for page 22
Getting results for page 23
Getting results for page 24
Getting results for page 25
Getting results for page 26
Getting results for page 27
Getting results for page 28
Getting results for page 29
Getting results for page 30
Getting results for page 31
Getting results for page 32
Getting results for page 33
Getting results for page 34
Getting results for page 35
Getting results for page 36
G

Getting results for page 288
Getting results for page 289
Getting results for page 290
Getting results for page 291
Getting results for page 292
Getting results for page 293
Getting results for page 294
Getting results for page 295
Getting results for page 296
Getting results for page 297
Getting results for page 298
Getting results for page 299
Getting results for page 300
Getting results for page 301
Getting results for page 302
Getting results for page 303
Getting results for page 304
Getting results for page 305
Getting results for page 306
Getting results for page 307
Getting results for page 308
Getting results for page 309
Getting results for page 310
Getting results for page 311
Getting results for page 312
Getting results for page 313
Getting results for page 314
Getting results for page 315
Getting results for page 316
Getting results for page 317
Getting results for page 318
Getting results for page 319
Getting results for page 320
Getting results for page 321
Getting result

['https://www.glassdoor.com/Reviews/GAIN-Capital-Reviews-E134569.htm',
 'https://www.glassdoor.com/Reviews/Mutual-of-America-Reviews-E20440.htm',
 'https://www.glassdoor.com/Reviews/Bank-of-China-New-York-Reviews-E260721.htm',
 'https://www.glassdoor.com/Reviews/United-Wholesale-Mortgage-UWM-Reviews-E699293.htm',
 'https://www.glassdoor.com/Reviews/Municipal-Credit-Union-Reviews-E235785.htm',
 'https://www.glassdoor.com/Reviews/World-Business-Lenders-Reviews-E624583.htm',
 'https://www.glassdoor.com/Reviews/National-Securities-Reviews-E310360.htm',
 'https://www.glassdoor.com/Reviews/PAYOMATIC-Reviews-E269496.htm',
 'https://www.glassdoor.com/Reviews/Scivantage-Reviews-E261326.htm',
 'https://www.glassdoor.com/Reviews/TrustCo-Bank-Corp-NY-Reviews-E1970.htm',
 'https://www.glassdoor.com/Reviews/Newedge-Reviews-E43338.htm',
 'https://www.glassdoor.com/Reviews/International-Fund-Services-Reviews-E318799.htm',
 'https://www.glassdoor.com/Reviews/AmeriSave-Mortgage-Reviews-E137772.htm',
 'h

In [33]:
with open('links_low_ratings.pickle','wb') as f:
    pickle.dump(all_links, f)

In [34]:
all_links

['https://www.glassdoor.com/Reviews/GAIN-Capital-Reviews-E134569.htm',
 'https://www.glassdoor.com/Reviews/Mutual-of-America-Reviews-E20440.htm',
 'https://www.glassdoor.com/Reviews/Bank-of-China-New-York-Reviews-E260721.htm',
 'https://www.glassdoor.com/Reviews/United-Wholesale-Mortgage-UWM-Reviews-E699293.htm',
 'https://www.glassdoor.com/Reviews/Municipal-Credit-Union-Reviews-E235785.htm',
 'https://www.glassdoor.com/Reviews/World-Business-Lenders-Reviews-E624583.htm',
 'https://www.glassdoor.com/Reviews/National-Securities-Reviews-E310360.htm',
 'https://www.glassdoor.com/Reviews/PAYOMATIC-Reviews-E269496.htm',
 'https://www.glassdoor.com/Reviews/Scivantage-Reviews-E261326.htm',
 'https://www.glassdoor.com/Reviews/TrustCo-Bank-Corp-NY-Reviews-E1970.htm',
 'https://www.glassdoor.com/Reviews/Newedge-Reviews-E43338.htm',
 'https://www.glassdoor.com/Reviews/International-Fund-Services-Reviews-E318799.htm',
 'https://www.glassdoor.com/Reviews/AmeriSave-Mortgage-Reviews-E137772.htm',
 'h

In [29]:
def get_num_pages(soup):
    results_string = soup.find('span',{'class':'resultCount'}).text
    num_results = int(results_string.split(' of ')[1].split(' Companies')[0]) 
    num_pages = math.ceil(int(num_results/10))
    return num_pages

In [30]:
def wait_for_page(driver):
    # WAIT FOR COMPANY PAGE TO LOAD 
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'Explore'))
        )
    except:
        print('error loading new page.')

In [31]:
"""
takes in soup object, master links list, and max rating
Companies rated below max rating included 
"""
def get_links_page(soup, all_links, max_rating):
    companies = soup.find_all('div', {'data-test':'employer-card-single'})
    for company in companies:
        # get info from each company 
        company_name = company.find('h2',{'data-test':'employer-short-name'}).text 
        rating = float(company.find('span',{'data-test':'rating'}).text)
        review_url = 'https://www.glassdoor.com' + company.find('a',{'data-test':'cell-Reviews-url'})['href']
        
        if rating < max_rating: 
            all_links.append(review_url)
            
    return all_links
